In [ ]:
# download txt from https://github.com/ceclinux/huffman/blob/master/莎士比亚全集英文版.txt and save it into data folder, use curl

import os
import numpy as np
import tqdm
!curl -o data/shakespeare.txt https://raw.githubusercontent.com/ceclinux/huffman/master/%E8%8E%8E%E5%A3%AB%E6%AF%94%E4%BA%9A%E5%85%A8%E9%9B%86%E8%8B%B1%E6%96%87%E7%89%88.txt

In [ ]:
# open the file and read the first 1000 characters
with open('data/shakespeare.txt', 'r') as file:
    text = file.read()
    print(text[:1000])

In [ ]:
import re
tokens = re.findall(r"[a-zA-Z]+|[^\s\w]|[\n]", text)
print(tokens[:100])

In [ ]:
# create a frequency table, from highest to lowest, and draw a bar chart
from collections import Counter
import matplotlib.pyplot as plt
frequency = Counter(tokens)
frequency = dict(sorted(frequency.items(), key=lambda x: x[1], reverse=True))

In [ ]:
plt.plot(frequency.values())
plt.semilogy()
plt.xlabel('Rank')
plt.ylabel('Frequency')

In [ ]:
vocab_size = 1000
frequency = dict(list(frequency.items())[:vocab_size])
transition_matrix = np.zeros((vocab_size, vocab_size), dtype=int)

for i in tqdm.tqdm(range(1, len(tokens))):
    left = tokens[i-1]
    right = tokens[i]
    if left in frequency and right in frequency:
        transition_matrix[list(frequency.keys()).index(left)][list(frequency.keys()).index(right)] += 1

In [ ]:
# generate according to the transition matrix
def generate(first='the', length=100):
    sentence = [first]
    index = list(frequency.keys()).index(first)
    for _ in range(length):
        next_index = np.random.choice(vocab_size, p=transition_matrix[index]/transition_matrix[index].sum())
        sentence.append(list(frequency.keys())[next_index])
        index = next_index
    return ' '.join(sentence)

print(generate('she'))

# RNN

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

In [ ]:
class RNNScratch(nn.Module):  #@save
    """The RNN model implemented from scratch."""
    def __init__(self, input_dim, hidden_dim, sigma=0.01):
        super().__init__()

        self.input_dim, self.hidden_dim = input_dim, hidden_dim
        self.W_ih = nn.Parameter(
            torch.randn(input_dim, hidden_dim) * sigma)
        self.W_hh = nn.Parameter(
            torch.randn(hidden_dim, hidden_dim) * sigma)
        self.b_h = nn.Parameter(torch.zeros(hidden_dim)) # equivalenet to b_ih + b_hh in the slides notation.


    def forward(self, inputs, state=None):
        # inputs shape: (num_steps, batch_size, input_dim)
        # state shape: (batch_size, hidden_dim)
        if state is None:
            # Initial state with shape: (batch_size, hidden_dim)
            state = torch.zeros((inputs.shape[1], self.hidden_dim),
                            device=inputs.device)
        else:
            state, = state
        outputs = []
        for X in inputs:  # Shape of inputs: (num_steps, batch_size, input_dim)
            state = torch.tanh(torch.matmul(X, self.W_ih) +
                            torch.matmul(state, self.W_hh) + self.b_h)
            outputs.append(state)
        return torch.stack(outputs), (state,)
    
rnn = RNNScratch(input_dim=16, hidden_dim=512)
    

# create a input sequence
LEN = 100 # sequence length
BATCH = 32 # batch size
DIM = 16 # input dimension
x = torch.randn(LEN, BATCH, DIM)
y, state = rnn(x)

In [ ]:
print("output shape:", y.shape) # sequence length * batch size * hidden dimension
print("state shape:", state[0].shape) # batch size * hidden dimension